In [ ]:
# from langchain.llms.openai import OpenAI # text-davinci-003 Model (사용 안 함)
from langchain.chat_models import ChatOpenAI # GPT-3.5-turbo Model (text-davinci-003의 1/10 가격)

# llm = OpenAI()
chat = ChatOpenAI(
    temperature=0.1 # 낮을 수록 일관성 있음, 높을 수록 무작위성(창의성) 있음, default 0.7
)

In [ ]:
##### 방식 1. HumanMessage, AIMessage, SystemMessage 사용 #####
from langchain.schema import HumanMessage, AIMessage, SystemMessage # HumanMessage: 사용자 입력, AIMessage: AI 출력, SystemMessage: 시스템 출력, prompt를 직접 입력할 때 사용

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}."),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}? Also, what is your name?")
]

chat.predict_messages(messages)

In [ ]:
##### 방식 2. PromptTemplate 사용 #####
from langchain.prompts import PromptTemplate # PromtTemplate은 string에서 template을 생성하기 위해 사용

template_c = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}? Also, what is your name?") # template 생성, {country_a}, {country_b}는 변수

prompt_a = template_c.format(country_a="Maxico", country_b="Thailand") # template에 변수를 적용하여 prompt 생성

chat.predict(prompt_a) # prompt_a를 입력하여 AI 출력 예측

In [ ]:
##### 방식 3. ChatPromptTemplate 사용 #####
from langchain.prompts import ChatPromptTemplate # CharPromptTemplate는 messages에서 template을 생성하기 위해 사용

template_b = ChatPromptTemplate.from_messages( # system, ai, human 순서로 messages를 ("speaker", "content") 형태로 입력, {language}, {name}, {country_a}, {country_b}는 변수
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, what is your name?")
    ]
)

prompt_b = template_b.format_prompt( # template에 변수를 적용하여 prompt 생성
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt_b) # prompt_b를 입력하여 AI 출력 예측

In [ ]:
##### 방식 4. ChatPromptTemplate 사용 및 사용자 정의 함수를 사용한 응용 #####
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser): # AI 출력을 파싱하기 위한 클래스, BaseOutputParser를 상속받아 구현, 파싱이란? AI 출력을 의미있는 형태로 변환하는 것
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

par = CommaOutputParser() # CommaOutputParser 객체 par 생성 (함수를 사용하기 위해 생성 필요함)

template_c = ChatPromptTemplate.from_messages( # {max_items}은 답변의 최대 개수, {question}은 사용자의 질문
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else."),
        ("human", "{question}")
    ]
)

prompt_c = template_c.format_messages( # template에 변수를 적용하여 prompt 생성
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt_c) # prompt_c를 입력하여 AI 출력 예측

par.parse(result.content)

In [ ]:
##### 방식 5. chain format을 사용한 응용 #####

chain = template_c | chat | par # template -> chat -> par 순서로 실행됨

chain.invoke(
    {
        "max_items": 10,
        "question": "What are the pokemons?"
    }
)

# 이런 방식으로도 조합할 수 있음
# chain_one = template1 | chat1 | parser1: template1 -> chat1 -> parser1 순서로 실행됨
# chain_two = template2 | chat2 | parser2: template2 -> chat2 -> parser2 순서로 실행됨
# all = chain_one | chain_two | ouput: chain_one -> chain_two -> output 순서로 실행됨